In [1]:
import numpy as np

In [2]:
WEAPONS = {("Redhorn", 1): (542, 0.882, 0.20, 0.28), 
           ("Redhorn", 5): (542, 0.882, 0.40, 0.56), 
           ("Serpent Spine", 1): (510, 0.276, 0, 0.30), 
           ("Serpent Spine", 5): (510, 0.276, 0, 0.50), 
           ("Whiteblind", 1): (510, 0.517, 0.24, 0), 
           ("Whiteblind", 5): (50, 0.517, 0.48, 0)}
SUBSTAT_LIST = ("HP%", "HP", "ATK%", "ATK", "DEF%", "DEF", "EM", "ER%", "CR%", "CD%")
SUBSTAT_BASE = [0.0496, 253.9, 0.0496, 16.54, 0.0620, 19.68, 19.82, 0.0551, 0.0331, 0.0662] 
#SUBSTAT_BASE = [0.058, 299, 0.058, 19, 0.073, 23, 23, 0.065, 0.039, 0.078] # These assume rolls are perfect
ITTO_ATK = 191
ITTO_DEF = 930
ITTO_CR = 0.242
ITTO_CD =  0.50
ITTO_A2 = 0.35
HUSK_DEF = 0.54
HUSK_GEO = 0.24
FEATHER = 311
HAT_CR = 0.311
HAT_CD = 0.622
SANDS = 0.583
GOBLET = 0.466

In [3]:
def calc_arti():
    totals = [20] # Possible liquid rolls
    nums = []
    for i in totals:
        total = i
        for i in range(2):
            val = np.random.randint(0, 8)
            nums.append(val)
            total -= val
        nums.append(total)
    liquidrolls = [2,2,2,2,2+nums[0],2,2,2,2+nums[1],2+nums[2]]
    substats = {SUBSTAT_LIST[i]: SUBSTAT_BASE[i]*liquidrolls[i] for i in range(len(SUBSTAT_LIST))} # Returns a dictionary with substat as key and multiplied rolls as value
    return substats

In [4]:
def calc_weap(weap, refine, buff, atk_buff):
    # -- Start of conditions
    if weap == "Redhorn":
        redhorn_def = WEAPONS[(weap, refine)][2]
        redhorn_bonus = WEAPONS[(weap, refine)][3]
    else:
        redhorn_def = redhorn_bonus = 0.00
    if weap == "Serpent Spine":
        ss_bonus = WEAPONS[(weap, refine)][3]
    else:
        ss_bonus = 0.00
    if weap == "Whiteblind":
        wb_atk = wb_def = WEAPONS[(weap, refine)][2]
    else:
        wb_atk = wb_def = 0.00
    if buff:
        geo_reso = 0.15
        gorou_c6 = 0.40
        gorou_def = 371
        gorou_geo = 0.15
        gorou_a4 = 0.25
        geo_shred = -0.15
    else:
        geo_reso, gorou_c6, gorou_def, gorou_geo, gorou_a4, geo_shred = 0.00, 0.00, 0, 0.00, 0.00, 0.00
    if atk_buff:
        noblesse = 0.20
        totm = 0.20
    else:
        noblesse = totm = 0.00
    # -- End of conditions
    arti_stats = calc_arti()
    base_atk = ITTO_ATK+WEAPONS[(weap, refine)][0]
    base_def = ITTO_DEF
    arti_atk = arti_stats["ATK"]+FEATHER
    total_atkp = arti_stats["ATK%"]+noblesse+totm+wb_atk
    total_atk = arti_atk+(base_atk*(1+total_atkp))
    arti_def = arti_stats["DEF"]
    total_defp = arti_stats["DEF%"]+HUSK_DEF+SANDS+gorou_a4+wb_def+redhorn_def+(WEAPONS[(weap, refine)][0] if weap == "Whiteblind" else 0.00)
    total_def = arti_def+(base_def*(1+total_defp))+gorou_def
    new_atk = (total_def*1.0368)+total_atk
    crit = arti_stats["CR%"]+ITTO_CR+(WEAPONS[(weap, refine)][1] if weap == "Serpent Spine" else 0.00)+(HAT_CR if weap == "Redhorn" else 0.00)
    cdmg = arti_stats["CD%"]+ITTO_CD+(WEAPONS[(weap, refine)][1] if weap == "Redhorn" else 0.00)+(HAT_CD if weap == "Serpent Spine" else 0.00)+gorou_c6
    if weap == "Whiteblind":
        if arti_stats["CR%"] < 0.15:
            crit += HAT_CR
        else:
            cdmg += HAT_CD
    else:
        pass
    geo_dmg = GOBLET+HUSK_GEO+gorou_geo
    extra_dmg = geo_reso+ss_bonus
    hit1 = (((new_atk*1.5662)+(total_def*(redhorn_bonus)))*(1+(geo_dmg+extra_dmg))*(1+(cdmg*crit)))*(0.487*(1-geo_shred))
    hit2 = (((new_atk*1.5096)+(total_def*(redhorn_bonus)))*(1+(geo_dmg+extra_dmg))*(1+(cdmg*crit)))*(0.487*(1-geo_shred))
    hit3 = (((new_atk*1.8115)+(total_def*(redhorn_bonus)))*(1+(geo_dmg+extra_dmg))*(1+(cdmg*crit)))*(0.487*(1-geo_shred))
    hit4 = (((new_atk*2.3172)+(total_def*(redhorn_bonus)))*(1+(geo_dmg+extra_dmg))*(1+(cdmg*crit)))*(0.487*(1-geo_shred))
    comboslash = (((new_atk*1.802)+(total_def*(redhorn_bonus+ITTO_A2)))*(1+(geo_dmg+extra_dmg))*(1+(cdmg*crit)))*(0.487*(1-geo_shred))
    finalslash = (((new_atk*3.774)+(total_def*(redhorn_bonus+ITTO_A2)))*(1+(geo_dmg+extra_dmg))*(1+(cdmg*crit)))*(0.487*(1-geo_shred))
    skill = ((new_atk*5.5296)*(1+(geo_dmg+extra_dmg))*(1+(cdmg*crit)))*(0.487*(1-geo_shred))
    rotation = skill + ((hit1+hit2+hit3+hit4)+hit1+hit2)+((4*comboslash)+(2*comboslash)+(2*finalslash))
    dps = rotation//20
    return dps

In [5]:
def ave_weap(weap, refine, buff, atk_buff, n):
    weap_sums = []
    for i in range(n):
        total_dps = calc_weap(weap, refine, buff, atk_buff)
        weap_sums.append(total_dps)
    print(f"Average DPS (20s) using {weap} R{refine} is {np.average(weap_sums)} with a standard deviation of {np.std(weap_sums)}.")

<b>Rotation</b>: Q -> N4 -> SCA5D -> E -> N2SCA3 (as provided by Jrm.Spirit)
<ul> 
    <li>20 fixed subs, 20 liquid subs optimized for <i>DEF%, CR%,</i> and <i>CD%</i>. (all perfect rolls)</li>
    <li>C0 Lvl 90 Itto 10/10/10 talents against Lvl 100 enemy with 10% universal resistance</li>
    <li> 4p Husk, max stacks, DEF%/Geo%/CR or CD
    <li> Buffs include C6 Gorou E at Lvl 10, Zhongli Shred, Geo Reso</li>
    <li> ATK Buffs are optional (NO, TotM)</li
    </ul>
    

In [6]:
simuls = 1000
isbuffed = True
isbuffed_atk = False
ave_weap("Serpent Spine", 5, isbuffed, isbuffed_atk, simuls)
ave_weap("Redhorn", 1, isbuffed, isbuffed_atk, simuls)

Average DPS (20s) using Serpent Spine R5 is 28897.432 with a standard deviation of 908.5695589089478.
Average DPS (20s) using Redhorn R1 is 29045.704 with a standard deviation of 992.8078839251832.
